# Willhaben - der österreichische Gebrauchtwagenmarkt

## Projekt Vorstellung

Willhaben.at kennt jeder (zumindest in Österreich)! :)

Willhaben.at ist eine der größten Marktplattformen in Österreich, auf denen Gebrauchtwagen gehandelt werden. Im Durchschnitt sind ca. 100.000 Autos gelistet (Zeitraum Oktober-November 2022). 

Unser Projekt verfolgt das Ziel einen Snapshot des Marktes zu machen und den Einfluss verschiedener Faktoren auf den Preis eines Autos zu analysieren. Die Fragestellungen sind nicht weltbewegend, aber die gesammelten Daten enthalten eine große Variabilität und sind eine ideale Übung, um Python zu lernen.



## 0. Bevor wir starten...

Wir erklären die Datenbeschaffung Schritt für Schritt im Jupyter Notebook, allerdings verweisen wir auch auf die Existenz folgender Python Skripte (machen das gleiche, nur ohne Notebook Overhead).

```sh
## in Shell ausführen:
$ python scripts/01scrapper.py
$ python scripts/02json_to_csv.py
$ python scripts/03validation.py
```

## 1. Web Scrapping

Willhaben stellt die Daten nicht als freien Download zur Verfügung (zB in Form einer CSV Datei), sondern arbeitet sie als Inserat auf der Page ein. Um die Daten zu erhalten, müssen wir die Daten programmatisch extrahieren ("scrappen"). 

!["Inserat"](./assets/inserat.png)


Als nächstes schauen wir uns die URL an, die folgende Struktur aufweist:

- https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse?page=50&rows=90&PRICE_TO=25000

Über Query Parameters (nach dem "?" in der URL) lässt sich steuern, welche Anzeigen auf Willhaben sichtbar sind. Man kann bis zu 999 Seiten anzeigen lassen (`page=999`). Über die Filter Funktion, zB "Preis bis zu", erfahren wir, dass `PRICE_TO` ebenfalls ein Parameter ist. Dieser ist erforderlich, um die Daten zu segmentieren, falls man mit dem theoretischen Limit von `page=999` und `rows=90` nicht auskommt (89.910 Inserate).

Der nächste Schritt ist zu analysieren, wie die Seite von Willhaben aufgebaut ist. Dazu schauen wir uns den Quellcode der HTML Seite an und stellen fest, dass tatsächlich die Daten als JSON mitgeliefert werden! Hintergrund ist der, dass Willhaben.at mit dem Web Framework Next.js aufgebaut ist: Der Browser ruft die URL auf, erhält JSON Daten und über JavaScript werden einzelne HTML Elemente dynamisch ausgetauscht. 

Im HTML Code ist das JSON hier zu finden: 
```html
<script id="__NEXT_DATA__" type="application/json">
  // json data...
</script>
```

Damit haben wir alle Wissenskomponenten, die wir brauchen, um die Daten zu extrahieren. Time to code!




Zum Scrappen, fahren wir keine schweren Geschütze auf (e.g. Playwright), sondern nutzen die Python Library `requests`. Außerdem definieren wir zwei eigene Exceptions, falls ein Request schief geht oder das JSON Fehler enthält.

In [1]:
import requests
import json
import time
from datetime import datetime
from typing import Dict

class JSONDataError(Exception):
    pass

class RequestNotSuccessfulError(Exception):
    pass

Requests ist sehr einfach, man schickt eine URL als Input und erhält eine Response als Output:

In [3]:
r = requests.get("https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse?page=50&rows=90&PRICE_TO=25000")
print(f"HTTP Status code: {r.status_code}")
# r.text        # der komplette HTML Code als Text

HTTP Status code: 200


Als Nächstes definieren wir eine Funktion, die mit Query Parameters gefüttert wird und als Output die Response des Willhaben Servers im Text Format schickt. Innerhalb der Funktion wird die URL dynamisch konstruiert und ein paar Cookies mitgeschickt (DSVGO Consent, etc.).

In [4]:
def get_html_from_willhaben(page: int = 1, price_to: int = 0, price_from: int = 0) -> str:
    """
    Scrap Willhaben Gebrauchtwagenboerse and return page html as string 
    """
    url = "https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse"
    cookies = {
        "IADVISITOR": "a58cf0e4-74a9-40c3-bf44-5fb0b9a714bc",
        "context": "prod",
        "TRACKINGID": "75349b84-607f-4161-9bc4-99f8aeddb123",
        "x-bbx-csrf-token": "5a4f4a0d-f121-4146-9f59-32ebcac0663d",
        "SRV": "1|Y0bcV",
        "didomi_token": "eyJ1c2VyX2lkIjoiMTgzY2NjY2EtYTMzZS02ZTA4LTkxOGYtMDNkOWYxYTIzM2U1IiwiY3JlYXRlZCI6IjIwMjItMTAtMTJUMTU6MjU6MTEuMjEyWiIsInVwZGF0ZWQiOiIyMDIyLTEwLTEyVDE1OjI1OjExLjIxMloiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiYW1hem9uIiwiZ29vZ2xlIiwiYzpvZXdhLVhBUW1HU2duIiwiYzphbWF6b24tbW9iaWxlLWFkcyIsImM6aG90amFyIiwiYzp1c2Vyem9vbSIsImM6YW1hem9uLWFzc29jaWF0ZXMiLCJjOnh4eGx1dHprLW05ZlFrUHRMIiwiYzpvcHRpb25hbGUtYm5BRXlaeHkiXX0sInB1cnBvc2VzIjp7ImVuYWJsZWQiOlsieHh4bHV0enItcWtyYXAzM1EiLCJnZW9sb2NhdGlvbl9kYXRhIiwiZGV2aWNlX2NoYXJhY3RlcmlzdGljcyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSIsImM6d2lsbGhhYmVuLVpxR242WXh6Il19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFrdUFFQUZrQkpZQS5Ba3VBQ0FrcyJ9",
        "RANDOM_USER_GROUP_COOKIE_NAME": "48",
        "euconsent-v2": "CPgvCMAPgvCMAAHABBENCkCsAP_AAH_AAAAAG9tf_X_fb2_j-_59f_t0eY1P9_7_v-0zjhedk-8Nyd_X_L8X52M7vB36pq4KuR4ku3LBAQdlHOHcTQmw6IkVqSPsbk2Mr7NKJ7PEmlMbOydYGH9_n1XT-ZKY79__f_7z_v-v___37r__7-3f3_vp9V-BugBJhq3EAXYljgTbRhFAiBGFYSHQCgAooBhaIDCAlcFOyuAn1hAgAQCgCMCIEOAKMGAQAAAQBIREAIEeCAAAEQCAAEACoRCAAjQBBQASBgEAAoBoWAEUAQgSEGRARFKYEBEiQUE8gQglB_oYYQh1FAAA.f_gAD_gAAAAA",
        "_pbjs_userid_consent_data": "3265244935257896",
        "COUNTER_FOR_ADVERTISING_FIRST_PARTY_UID_V2": "0",
    }
    params = {"rows": 75}

    if page > 0 and type(page) == int:
        params.update({"page": page})

    if price_to > 0 and type(price_to) == int:
        params.update({"PRICE_TO": price_to})

    if price_from > 0 and type(price_from) == int:
        params.update({"PRICE_FROM": price_from})

    r = requests.get(url, cookies=cookies, params=params, timeout=60)

    if not r.ok:
        raise RequestNotSuccessfulError("Request was not successful", page)

    return r.text

Aus dem HTML Text extrahieren wir die JSON Datei, in dem wir nach dem Script Tag mit `id='__Next_DATA__'` suchen. Dort benötigen wir nicht alles, sondern nur das Feld `advertSummaryList`.  

In [5]:
def extract_json_from_html(html: str) -> Dict:
    """
    Extract JSON from Next.js page (Willhaben), by searching for id=NEXT_DATA
    """
    if "__NEXT_DATA__" not in html:
        raise JSONDataError("No script tag with id='__NEXT_Data__'")

    if "advertSummaryList" not in html:
        raise JSONDataError("No advertSummaryList in JSON Data")    

    script_tag_open = """<script id="__NEXT_DATA__" type="application/json">"""
    script_tag_close = """</script>"""
    start = html.find(script_tag_open) + len(script_tag_open)
    end = start + html[start:].find(script_tag_close)

    json_data = json.loads(html[start:end])
    data = json_data["props"]["pageProps"]["searchResult"]["advertSummaryList"]

    if len(data["advertSummary"]) == 0:
        raise JSONDataError("Empty advertSummary")

    return data

Zum Schluss eine letzte Funktion, die die obigen Funktionen in eine Schleife verpackt, bei Fehlern es neu versucht und die JSONs abspeichert. 

In [7]:
def loop_scrap_save(pages: int = 999, price_to: int = 0, price_from: int = 0):
    """
    Set Price Filter, loop through all pages and save scrapped json
    """
    now = datetime.now().strftime("%Y-%m-%d")

    for page in range(1, pages+1):
        time.sleep(0.5)
        for retry in range(5):
            try:
                html = get_html_from_willhaben(page, price_to, price_from)
                data = extract_json_from_html(html)

                if price_from > 0:
                    file_name = f"./json/{now}_page={page}-price_from_{price_from}.json"

                if price_to > 0: 
                    file_name = f"./json/{now}_page={page}-price_to_{price_to}.json"

                with open(file_name, "w", encoding="utf-8") as file:
                    json.dump(data, file, indent=2, ensure_ascii=False)
                    print(f"Page: {page}, saved to {file_name}")
                
                break  # break retry loop

            except RequestNotSuccessfulError as e:
                print(e)
                time.sleep(10 + 10*retry)

            except JSONDataError as e:
                print(e)
                break

Scrapping wird gestartet mit:

In [9]:
# Scrap 3 sample pages to demonstrate code
try:
    loop_scrap_save(pages=3, price_to=24999)
    loop_scrap_save(pages=3, price_from=24999)
except KeyboardInterrupt:
    print("Code stopped with CTRL+C")

Page: 1, saved to ./json/2022-11-16_page=1-price_to_24999.json
Page: 2, saved to ./json/2022-11-16_page=2-price_to_24999.json
Page: 3, saved to ./json/2022-11-16_page=3-price_to_24999.json
Page: 1, saved to ./json/2022-11-16_page=1-price_from_24999.json
Page: 2, saved to ./json/2022-11-16_page=2-price_from_24999.json
Page: 3, saved to ./json/2022-11-16_page=3-price_from_24999.json


## 2. JSON zu CSV umwandeln

Die gespeicherten JSON File müssen wir nun in ein gesammeltes CSV File umwandeln. Dazu ein paar Vorbereitungen:

In [1]:
import json
import csv
from pathlib import Path
from typing import Dict
import gc

Als nächstes suchen wir uns die Feldnamen aus, die wir extrahieren wollen und erstellen eine weitere Liste mit den bereinigten Namen. 

In [2]:
SCHEMA = [
    "id", "description", "PRODUCT_ID", "HEADING", "BODY_DYN", "PRICE", "YEAR_MODEL",
    "MILEAGE", "CAR_MODEL/MAKE", "CAR_MODEL/MODEL", "CAR_TYPE", "NO_OF_OWNERS",
    "NOOFSEATS", "ENGINE/EFFECT", "ENGINE/FUEL_RESOLVED", "TRANSMISSION_RESOLVED",
    "CONDITION_RESOLVED", "WARRANTY_RESOLVED", "PUBLISHED_String", "COUNTRY",
    "COORDINATES", "POSTCODE", "STATE", "DISTRICT", "ADDRESS", "LOCATION",
    "ORGNAME", "fnmmocount", "UPSELLING_AD_SEARCHRESULT", "ISPRIVATE", "EQUIPMENT_RESOLVED",
]
clean_names = SCHEMA[:]
clean_names[8] = "brand"
clean_names[9] = "model"
clean_names[13] = "engine_effect"
clean_names[14] = "engine_fuel_resolved"
clean_names = [item.lower() for item in clean_names]

Dann schreiben wir eine Funktion, die ein Element aus der JSON Struktur in eine Zeile für das CSV File umwandelt. 


In [3]:
def get_row_from_json_item(item: Dict) -> Dict:
    """
    Extract relevant data from Willhaben JSON element and return a row dict
    """
    row = {"id": item["id"], "description": item["description"]}
    for el in item["attributes"]["attribute"]:
        if el["name"] in SCHEMA:
            name = el["name"]
            value = el["values"][0]
            if name in "EQUIPMENT_RESOLVED":
                value = "|".join(el["values"])
            row.update({name: value})
    return row

In der nächsten Code Cell erstellen wir die CSV Datei. Dabei benutzen wir zwei Schleifen, einmal für die JSON files und einmal für das Array innerhalb des JSONs. Die Zeilen werden nach und nach ins CSV File geschrieben. Die Code Cell benötigt ein paar Minuten.

In [7]:
# Open CSV File and prep it
csv_file = open("./data/data_.csv", "w", newline="", encoding="utf-8")
writer = csv.DictWriter(csv_file, dialect="excel", delimiter=";", fieldnames=SCHEMA)
writer.writeheader()

# Go through json folder and iterate through all json files
for file in Path("./json").iterdir():
    with open(file.resolve(), "r", encoding="utf-8") as f:
        print(f"File: {file.stem}")
        data = json.load(f)["advertSummary"]
        for item in data:
            row = get_row_from_json_item(item)
            writer.writerow(row)
        gc.collect()

# Close CSV File
csv_file.close()

File: 2022-10-17_page=1-price_from_25000
File: 2022-10-17_page=1-price_to_24999
File: 2022-10-17_page=10-price_from_25000
File: 2022-10-17_page=10-price_to_24999
File: 2022-10-17_page=100-price_from_25000
File: 2022-10-17_page=100-price_to_24999
File: 2022-10-17_page=101-price_from_25000
File: 2022-10-17_page=101-price_to_24999
File: 2022-10-17_page=102-price_from_25000
File: 2022-10-17_page=102-price_to_24999
File: 2022-10-17_page=103-price_from_25000
File: 2022-10-17_page=103-price_to_24999
File: 2022-10-17_page=104-price_from_25000
File: 2022-10-17_page=104-price_to_24999
File: 2022-10-17_page=105-price_from_25000
File: 2022-10-17_page=105-price_to_24999
File: 2022-10-17_page=106-price_from_25000
File: 2022-10-17_page=106-price_to_24999
File: 2022-10-17_page=107-price_from_25000
File: 2022-10-17_page=107-price_to_24999
File: 2022-10-17_page=108-price_from_25000
File: 2022-10-17_page=108-price_to_24999
File: 2022-10-17_page=109-price_from_25000
File: 2022-10-17_page=109-price_to_2499

In einem zweiten Schritt erstellen wir ein CSV mit bereinigten Header, wobei die Daten erneut in das CSV File geschrieben werden. Der zweite Schritt ist verschwenderisch, aber auf die schnelle haben wir keine bessere Lösung gefunden, zumal der Code sowieso nur einmalig ausgeführt wird. 

In [8]:
# Rewrite header with clean names
with open("./data/data_.csv", "r", newline="", encoding="utf-8") as input, open(
        "./data/data.csv", "w", newline="", encoding="utf-8") as output:
    reader = csv.reader(input, delimiter=";")
    writer = csv.writer(output, delimiter=";")

    header = next(reader)
    writer.writerow(clean_names)

    for row in reader:
        writer.writerow(row)

# Delete first csv file
Path("./data/data_.csv").unlink()

### 3. Datenvalidierung

Bevor wir mit den Daten arbeiten, führen wir eine leichte Datenvalidierung mit `pydantic` durch. Vorbereitung:

In [9]:
from pydantic import BaseModel, Field, ValidationError, validator
from typing import Union
import csv

Im Prinzip wollen wir nur validieren, dass bestimmte numerische Features (Preis, Km-Stand, Baujahr...) zumindest einen Wert enthalten und größer als Null sind. Außerdem soll die mitgelieferte ID von Willhaben ebenfalls einen neunstelligen Code enthalten, um sicher zu gehen, dass wir Inserate gescrappt haben. 

In [11]:
class Inserat(BaseModel):
    id: str = Field(min_length=9)
    description: Union[str, None]
    product_id: Union[int, None]
    heading: str
    body_dyn: Union[str, None]
    price: int = Field(ge=0)
    year_model: int = Field(ge=0)
    mileage: int = Field(ge=0)
    brand: Union[str, None]
    model: Union[str, None]
    car_type: Union[str, None]
    no_of_owners: Union[str, None]
    noofseats: Union[str, None]
    engine_effect: int = Field(ge=0)
    engine_fuel_resolved: Union[str, None]
    transmission_resolved: Union[str, None]
    condition_resolved: Union[str, None]
    warranty_resolved: Union[str, None]
    published_string: Union[str, None]
    country: Union[str, None]
    coordinates: Union[str, None]
    postcode: Union[str, None]
    state: Union[str, None]
    district: Union[str, None]
    address: Union[str, None]
    location: Union[str, None]
    orgname: Union[str, None]
    fnmmocount: Union[int, None]
    upselling_ad_searchresult: Union[str, None]
    isprivate: Union[int, None]
    equipment_resolved: Union[str, None]

    @validator("year_model", "mileage", "engine_effect", "price", pre=True, allow_reuse=True)
    def make_int(cls, v):
        return int(v)

Wir lesen den Rohdatensatz ein und durchlaufen jede Zeile mit einer Schleife. Jede Zeile wird validiert, in dem sie durch das Pydantic Model geschleußt wird. Sollte die Zeile korrekt sein, wird sie der `validated_data` Liste hinzugefügt. Als letzten Schritt schreiben wir die Daten wieder in ein CSV File. 

In [12]:
validated_data = []
with open("./data/data.csv", "r", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    data = list(reader)
    for item in data:
        try:
            validated_data.append(Inserat(**item).dict())
        except ValidationError as e:
            print(item["id"], e)

print(f"Rows of raw data {len(data)}")
print(f"Rows of validated data {len(validated_data)}")

with open("./data/data-validated.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, delimiter=";", dialect="excel",
            fieldnames=list(validated_data[0].keys()),
        )
    writer.writeheader()
    for item in validated_data:
        writer.writerow(item)

597837549 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
597836371 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
603077567 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
603077652 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
603077657 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
612162272 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
612152059 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
579125584 1 validation error for Inserat
year_model
  invalid literal for int() with base 10: '' (type=value_error)
591899499 1 validation error for Inserat
year_model
  invalid literal fo